In [16]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import time

import sys
sys.path.append('../')
from src.parameters import STR_CONN
from src.utils import download_data_stocks, download_data_ticker

In [14]:
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)

In [15]:
stock_symbols = pd.read_excel('../data/tickers.xlsx')
stock_symbols = stock_symbols['Símbolo'].to_list()

In [2]:
# stock_symbols = ['META']

In [22]:
bad_symbol = []
for stock_symbol in stock_symbols:
    try:
        ticker = download_data_ticker(stock_symbol)
        stock_data = download_data_stocks(stock_symbol, start_date, end_date)
        try:
            ticker.to_sql('stocks', con=STR_CONN, if_exists='append', index=False)
            stock_data.to_sql('stock_prices', con=STR_CONN, if_exists='append', index=False)
        except Exception as e:
            print('Not upload in database: ', stock_symbol)
            print(e)
    except Exception as e:
        print(stock_symbol, e)
        bad_symbol.append(stock_symbol)
    time.sleep(1)

[*********************100%***********************]  1 of 1 completed


In [23]:
bad_symbol

[]

In [25]:
# Descargar datos del índice S&P 500
sp500 = yf.Ticker("^GSPC")  # El símbolo del S&P 500

# Obtener los componentes del índice S&P 500 (principales acciones)
sp500_stocks = sp500.history(period="1d")  # Solo obtienes datos de precios
sp500_stocks

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-09-13 00:00:00-04:00,5603.339844,5636.27002,5601.649902,5626.02002,3500790000,0.0,0.0
